In [1]:
import pandas
import pickle

merged_data = pandas.read_pickle('../data/merged_data.pickle')
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268


In [2]:
import math

# analyze the max weight of a course
assessment_lists_per_student_per_course = merged_data.groupby(['code_module', 'code_presentation']).agg({
    "assessments": lambda x: list(x),
}).reset_index()

def analyze_max_weight_per_course_per_presentation(row):
    asessments_of_students = row['assessments']
    max_weight = 0
    for asessments_of_student in asessments_of_students:
        weights = [weight for (id_assessment, date_submitted, score, assessment_type, date, weight) in asessments_of_student]
        student_max_weight = sum(weights)
        if student_max_weight > max_weight:
            max_weight = student_max_weight
        

    row['max_weight'] = max_weight
    return row

max_weights_per_course_per_presentation = assessment_lists_per_student_per_course.apply(analyze_max_weight_per_course_per_presentation, axis = 1)
max_weights_per_course_per_presentation = max_weights_per_course_per_presentation.drop('assessments', axis=1).set_index(['code_module', 'code_presentation'])
max_weights_per_course_per_presentation.head(21)

max_weight
code_module code_presentation            
AAA         2013J                   100.0
            2014J                   100.0
BBB         2013B                   100.0
            2013J                   100.0
            2014B                   100.0
            2014J                   100.0
CCC         2014B                   200.0
            2014J                   200.0
DDD         2013B                   200.0
            2013J                   200.0
            2014B                   200.0
            2014J                   200.0
EEE         2013J                   100.0
            2014B                   100.0
            2014J                   100.0
FFF         2013B                   100.0
            2013J                   100.0
            2014B                   100.0
            2014J                   100.0
GGG         2013J                     0.0
            2014B                     0.0

In [3]:
def analyze_average_grade(assessments):
    total_weight = 0
    total_score = 0

    for assessment in assessments:
        id_assessment, date_submitted, score, assessment_type, date, weight = assessment
        if (not(type(score) == float)):
            print(score)
        total_weight +=  weight
        total_score += score * weight


    if total_weight > 0:
        average_score = total_score / total_weight
        return average_score
    else:
        return math.nan

merged_data['average_score'] = merged_data['assessments'].transform(analyze_average_grade)
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,average_score
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,82.4
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,65.4
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,76.3
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,55.0
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268,66.9


In [4]:
# jort
# sum of all clicks

def sum_clicks_vles(vles):
    sum_clicks = 0
    for [id_site, date, activity_type, sum_click] in vles:
        sum_clicks += sum_click
    return sum_clicks

merged_data["sum_clicks"] = merged_data["vles"].transform(sum_clicks_vles)
merged_data.head()

,code_module,code_presentation,id_student,assessments,vles,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,average_score,sum_clicks
0,AAA,2013J,11391,"[[1752, 18, 78.0, TMA, 19.0, 10.0], [1753, 53,...","[[546614, -5, homepage, 7], [546614, 0, homepa...",M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,82.4,934
1,AAA,2013J,28400,"[[1752, 22, 70.0, TMA, 19.0, 10.0], [1753, 52,...","[[546614, -10, homepage, 11], [546614, -8, hom...",F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,65.4,1435
2,AAA,2013J,31604,"[[1752, 17, 72.0, TMA, 19.0, 10.0], [1753, 51,...","[[546614, -10, homepage, 11], [546614, -9, hom...",F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,76.3,2158
3,AAA,2013J,32885,"[[1752, 26, 69.0, TMA, 19.0, 10.0], [1753, 75,...","[[546614, -10, homepage, 13], [546614, -9, hom...",F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,55.0,1034
4,AAA,2013J,38053,"[[1752, 19, 79.0, TMA, 19.0, 10.0], [1753, 64,...","[[546614, -10, homepage, 13], [546614, -9, hom...",M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268,66.9,2445
